In [1]:
import os, sys
sys.path.append("../../common/")
from python_tools import *

python tools loaded.


In [2]:
root_filenames_CV   = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/CVSet/*/*/sampler*.root")
root_filenames_Set2 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set2/*/*/sampler*.root")
root_filenames_Set3 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set3/*/*/sampler*.root")
root_filenames_Set4 = glob.glob("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set4/*/*/sampler*.root")

In [3]:
def print_qrun_strings(files,fdset):
    run_dict = {"Run1":set(), "Run3":set()}
    for f in files:
        dset = f.split("/")[-3]
        run = f.split("/")[-2].split("_")[0]
        #print(dset,run)
        run_dict[dset].add(run)
    #print(run_dict["Run1"])
    #print(run_dict["Run3"])

    run1_str = "query_set%d_run1_fd = '"%fdset
    for i,run in enumerate(run_dict["Run1"]):
        run1_str = run1_str+"run=%s"%run
        if i<(len(run_dict["Run1"])-1):
            run1_str = run1_str+" or "
        else:
            run1_str = run1_str+"'"
    print(run1_str)
    run3_str = "query_set%d_run3_fd = '"%fdset
    for i,run in enumerate(run_dict["Run3"]):
        run3_str = run3_str+"run=%s"%run
        if i<(len(run_dict["Run3"])-1):
            run3_str = run3_str+" or "
        else:
            run3_str = run3_str+"'"
    print(run3_str)
#print('query_set2_run1_fd = "run=32043865 or run=32043865")

In [4]:
print_qrun_strings(root_filenames_Set2,2)
print_qrun_strings(root_filenames_Set3,3)
print_qrun_strings(root_filenames_Set4,4)


query_set2_run1_fd = 'run=32043925 or run=32043865 or run=32043990'
query_set2_run3_fd = 'run=32044041 or run=32044121 or run=32044093'
query_set3_run1_fd = 'run=31851543 or run=31851544 or run=31851561'
query_set3_run3_fd = 'run=7265654 or run=7265677 or run=31873900'
query_set4_run1_fd = 'run=7240698 or run=7240700 or run=7240694'
query_set4_run3_fd = 'run=7240701 or run=7240703 or run=7240702'


In [5]:
pot_df_fd1 = pd.read_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs.h5","pot_df")
ev_df_fd1 = pd.read_hdf("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/mcana_dfs.h5","ev_df")

In [6]:
pot_df_fd1

totpot    totgoodpot  totspills  goodspills
run     subrun                                                   
6693842 6018    2.259947e+16  2.259947e+16          0           0
6693859 715     1.821702e+16  1.821702e+16          0           0
6693842 1640    2.185295e+16  2.185295e+16          0           0
        9210    2.039070e+16  2.039070e+16          0           0
        4969    1.777037e+16  1.777037e+16          0           0
...                      ...           ...        ...         ...
7165574 3881    6.524193e+15  6.524193e+15          0           0
        8666    1.464675e+16  1.464675e+16          0           0
        9978    3.089360e+16  3.089360e+16          0           0
        2269    2.037520e+16  2.037520e+16          0           0
        7591    3.474616e+16  3.474616e+16          0           0

[28537 rows x 4 columns]

In [7]:
ev_df_fd1.index.get_level_values('run').array

<PandasArray>
[6693842, 6693842, 6693842, 6693842, 6693842, 6693842, 6693842, 6693842,
 6693842, 6693842,
 ...
 7165574, 7165574, 7165574, 7165574, 7165574, 7165574, 7165574, 7165574,
 7165574, 7165574]
Length: 466225, dtype: uint64

In [8]:
rs_df_reco2 = pd.read_csv("/Users/wketchum/Data/MicroBooNE/FakeData2020/Set1/Set1_Run1_RSub.txt",names=["run","subrun"],header=None,sep=".")

In [9]:
rs_df_reco2

,run,subrun
0,6693842,389
1,6693842,1841
2,6693859,2632
3,6693842,3983
4,6693842,4252
...,...,...
12849,6693842,5715
12850,6693859,2349
12851,6693859,1595
12852,6693859,2922


In [10]:
rs_df_reco2_pot = pd.merge(rs_df_reco2,pot_df_fd1,left_on=["run","subrun"],right_on=["run","subrun"],how='left')

In [11]:
#ev_df_fd1 = ev_df_fd1.reset_index()
#ev_df_fd1 = ev_df_fd1.astype({"run":"int64","subrun":"int64","event":"int64"})
#pot_df_fd1 = pot_df_fd1.reset_index()
#pot_df_fd1 = pot_df_fd1.astype({"run":"int64","subrun":"int64"})

index1 = pd.MultiIndex.from_arrays([rs_df_reco2[col] for col in ['run', 'subrun']])
index2 = pd.MultiIndex.from_arrays([ev_df_fd1.index.get_level_values('run').array,
                                    ev_df_fd1.index.get_level_values('subrun').array])
#index3 = pd.MultiIndex.from_arrays([pot_df_fd1[col] for col in ['run', 'subrun']])

In [12]:
print(len(ev_df_fd1.loc[index2.isin(index1)]),len(ev_df_fd1))
#print(len(pot_df_fd1.loc[index3.isin(index1)]),len(pot_df_fd1))


164986 466225


In [13]:
print(index2.isin(index1))
print(index1)
print(index2)

[ True  True  True ... False False False]
MultiIndex([(6693842,  389),
            (6693842, 1841),
            (6693859, 2632),
            (6693842, 3983),
            (6693842, 4252),
            (6693842, 1716),
            (6693842, 4085),
            (6693859, 2477),
            (6693842,  430),
            (6693842, 1738),
            ...
            (6693859, 1926),
            (6693859,  248),
            (6693842, 9303),
            (6693842, 7330),
            (6693859, 2755),
            (6693842, 5715),
            (6693859, 2349),
            (6693859, 1595),
            (6693859, 2922),
            (6693859, 1292)],
           names=['run', 'subrun'], length=12854)
MultiIndex([(6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            (6693842, 6018),
            ...
     

In [14]:
rs_df_reco2_pot["totpot"].sum()

2.070220087022432e+20

In [15]:
pot_df_fd1 = pd.merge(pot_df_fd1,rs_df_reco2,left_on=["run","subrun"],right_on=["run","subrun"],how='right')

In [16]:
pot_df_fd1

,run,subrun,totpot,totgoodpot,totspills,goodspills
0,6693842,6018,2.259947e+16,2.259947e+16,0,0
1,6693842,1640,2.185295e+16,2.185295e+16,0,0
2,6693842,9210,2.039070e+16,2.039070e+16,0,0
3,6693842,4969,1.777037e+16,1.777037e+16,0,0
4,6693859,2554,2.322413e+16,2.322413e+16,0,0
...,...,...,...,...,...,...
12850,6693859,12,2.916704e+16,2.916704e+16,0,0
12851,6693859,2639,2.084553e+16,2.084553e+16,0,0
12852,6693859,1918,2.506222e+16,2.506222e+16,0,0
12853,6693859,2793,2.669451e+16,2.669451e+16,0,0


In [68]:
ev_df_fd1.sort_index().loc[([7165574,6693859],slice(None),[1,3],slice(None))]

n_particles  origin  mode  interaction_type  \
run     subrun event truth_index                                                
6693859 1      1     0                     11       1     0              1002   
               3     0                      9       1     0              1001   
        2      1     0                     17       1     0              1001   
               3     0                      8       1     0              1001   
        3      1     0                     12       1    10              1000   
...                                       ...     ...   ...               ...   
7165574 9997   3     0                     11       1     1              1013   
        9998   1     0                     11       1     0              1002   
               3     0                      9       1     0              1002   
        9999   1     0                      8       1     0              1001   
               3     0                     11       1    10              1000   

                                  ccnc      target    hit_nucl  hit_quark  \
run     subrun event truth_index                                            
6693859 1      1     0               1  1000180400        2112          0   
               3     0               0  1000180400        2112          0   
        2      1     0               0  1000180400        2112          0   
               3     0               0  1000180400        2112          0   
        3      1     0               0  1000180400  2000000201          0   
...                                ...         ...         ...        ...   
7165574 9997   3     0               1  1000180400        2212          0   
        9998   1     0               1  1000180400        2112          0   
               3     0               1  1000180400        2112          0   
        9999   1     0               0  1000180400        2112          0   
               3     0               1  1000260560  2000000201          0   

                                  hadronic_mass  interaction_x  ...  \
run     subrun event truth_index                                ...   
6693859 1      1     0                 0.886579       1.184377  ...   
               3     0                 1.012835       0.249322  ...   
        2      1     0                 1.090911       0.707626  ...   
               3     0                 0.918900       1.174875  ...   
        3      1     0                 0.983126       0.525599  ...   
...                                         ...            ...  ...   
7165574 9997   3     0                 1.200820       0.097383  ...   
        9998   1     0                 0.869120       1.371323  ...   
               3     0                 0.931260       1.089283  ...   
        9999   1     0                 0.998146       0.408805  ...   
               3     0                 1.089617       0.437646  ...   

                                  end_e_gamma  ke_gamma  n_mu  n_e  n_p_40MeV  \
run     subrun event truth_index                                                
6693859 1      1     0                    NaN       NaN   0.0  0.0        0.0   
               3     0                    NaN       NaN   1.0  0.0        1.0   
        2      1     0                    NaN       NaN   1.0  0.0        2.0   
               3     0                    NaN       NaN   1.0  0.0        1.0   
        3      1     0                    NaN       NaN   1.0  0.0        0.0   
...                                       ...       ...   ...  ...        ...   
7165574 9997   3     0                    NaN       NaN   0.0  0.0        1.0   
        9998   1     0                    NaN       NaN   0.0  0.0        1.0   
               3     0                    NaN       NaN   0.0  0.0        0.0   
        9999   1     0                    NaN       NaN   1.0  0.0        1.0   
               3     0                    NaN       NaN   0.0  0.0        1.0   



In [64]:
ev_df_fd1.index

MultiIndex([(6693842, 6018,  1, 0),
            (6693842, 6018,  2, 0),
            (6693842, 6018,  3, 0),
            (6693842, 6018,  4, 0),
            (6693842, 6018,  5, 0),
            (6693842, 6018,  6, 0),
            (6693842, 6018,  7, 0),
            (6693842, 6018,  8, 0),
            (6693842, 6018,  9, 0),
            (6693842, 6018, 10, 0),
            ...
            (7165574, 7591, 15, 0),
            (7165574, 7591, 16, 0),
            (7165574, 7591, 17, 0),
            (7165574, 7591, 18, 0),
            (7165574, 7591, 19, 0),
            (7165574, 7591, 20, 0),
            (7165574, 7591, 21, 0),
            (7165574, 7591, 22, 0),
            (7165574, 7591, 23, 0),
            (7165574, 7591, 24, 0)],
           names=['run', 'subrun', 'event', 'truth_index'], length=466225)